# 2020/02/07 - Exam

## Part 1 - MCQ

1. The false option is b. The column 'features' actually contains the vectors of features of the data set.
2. The blocks needed are 5. `data2019.txt` needs 2 blocks, `data2018.txt` needs 1 and `data2017.txt` needs 2.

## Part 2 - Programs

### 1

In [40]:
# With DFs

import sys

# out_path = sys.argv[1]
# in_path = sys.argv[2]

out_path = '/user/s315054/exam_sim/example_7Feb/ex1_DF'
in_path = '/data/students/bigdata_internet/exam_examples_data/exam20200207_data/Visualizations_Purchases.txt'

in_DF = spark.read.load(in_path, format='csv', sep=',', header=False, inferSchema=True)\
                .toDF('timestamp', 'username', 'itemID', 'purchased')

in_DF.createOrReplaceTempView('first_table')

23/01/31 15:12:21 WARN analysis.SimpleFunctionRegistry: The function is2019 replaced a previously registered function.
23/01/31 15:12:21 WARN analysis.SimpleFunctionRegistry: The function translpurch replaced a previously registered function.


In [43]:
# Filter for 2019 only
# Timestamp format: %Y/%m/%d-%h:%M:%S
spark.udf.register('is2019', lambda ts: ts.startswith('2019'))

def translPurch(item):
    if item == True:
        return 1
    elif item == False:
        return 0

spark.udf.register('translPurch', translPurch)

tmp1_DF = spark.sql("""
                SELECT username, count(*) AS n_views, sum(CAST(purchased == True AS int)) AS n_purch
                FROM first_table
                WHERE is2019(timestamp) == True
                GROUP BY username
""")

out1_DF = tmp1_DF.filter('n_views > 50').selectExpr('username', 'n_purch/n_views AS purch_rate').filter('purch_rate > 0.0005')

out1_DF.write.csv(out_path, header=False, sep=',')


23/01/31 15:15:41 WARN analysis.SimpleFunctionRegistry: The function is2019 replaced a previously registered function.
23/01/31 15:15:41 WARN analysis.SimpleFunctionRegistry: The function translpurch replaced a previously registered function.


In [80]:
!hdfs dfs -rm -r /user/s315054/exam_sim/example_7Feb/ex1_DF

23/01/31 16:05:43 INFO fs.TrashPolicyDefault: Moved: 'hdfs://BigDataHA/user/s315054/exam_sim/example_7Feb/ex1_DF' to trash at: hdfs://BigDataHA/user/s315054/.Trash/Current/user/s315054/exam_sim/example_7Feb/ex1_DF


In [81]:
# Using SparkSQL methods

out1_DF = in_DF.filter('is2019(timestamp) == True').withColumn('purchased', in_DF.purchased.cast('int'))\
            .groupBy('username').agg({'*':'count', 'purchased':'sum'}).withColumnRenamed('count(1)', 'n_views')\
            .withColumnRenamed('sum(purchased)', 'n_purchases').filter('n_views > 50')\
            .selectExpr('username', 'n_purchases/n_views AS purch_rate').filter('purch_rate > 0.0005')

out1_DF.write.csv(out_path, header=False, sep=',')

In [82]:
out1_DF.show()

+--------+-------------------+
|username|         purch_rate|
+--------+-------------------+
|  User30|0.38461538461538464|
+--------+-------------------+



### Ex 2a

In [114]:
!hdfs dfs -rm -r out_2a = /user/s315054/exam_sim/example_7Feb/ex2a_DF

rm: `out_2a': No such file or directory
rm: `=': No such file or directory
23/01/31 16:52:44 INFO fs.TrashPolicyDefault: Moved: 'hdfs://BigDataHA/user/s315054/exam_sim/example_7Feb/ex2a_DF' to trash at: hdfs://BigDataHA/user/s315054/.Trash/Current/user/s315054/exam_sim/example_7Feb/ex2a_DF


In [115]:
# DF
in_file = '/data/students/bigdata_internet/exam_examples_data/exam20200207_data/Visualizations_Purchases.txt'
out_2a = '/user/s315054/exam_sim/example_7Feb/ex2a_DF'

in_DF = spark.read.load(in_path, format='csv', sep=',', header=False, inferSchema=True)\
                .toDF('timestamp', 'username', 'itemID', 'purchased')

in_DF.createOrReplaceTempView('start_table')

spark.udf.register('getYear', lambda ts: int(ts.split('/')[0]))

filt_DF = spark.sql("""
                SELECT itemID
                FROM start_table
                WHERE getYear(timestamp) == 2019 OR getYear(timestamp) == 2018
                GROUP BY itemID
                HAVING sum(CAST(getYear(timestamp) == 2019 AS int)) >= 20  AND sum(CAST(getYear(timestamp) == 2018 AS int)) == 0
""")

filt_DF.write.csv(out_2a, header=False)

23/01/31 16:52:45 WARN analysis.SimpleFunctionRegistry: The function getyear replaced a previously registered function.


In [116]:
filt_DF.show()

+------+
|itemID|
+------+
|  ID28|
+------+



In [117]:
# Check results
spark.sql("""
            SELECT itemID, sum(CAST(getYear(timestamp) == 2019 AS int)) AS n_2019, sum(CAST(getYear(timestamp) == 2018 AS int)) AS n_2018
            FROM start_table
            GROUP BY itemID
""").show()

+------+------+------+
|itemID|n_2019|n_2018|
+------+------+------+
|  ID62|     1|     0|
|  ID25|     2|     0|
|  ID34|     2|     0|
|  ID54|     1|     0|
|   ID1|     1|     0|
|  ID84|     1|     0|
|  ID23|     1|     0|
|  ID58|     1|     0|
|  ID77|     1|     0|
|  ID35|     1|     0|
|  ID70|     1|     1|
|  ID43|     1|     0|
|  ID89|     1|     0|
|  ID11|     2|     0|
|  ID65|     1|     0|
|  ID28|    23|     0|
|  ID56|     1|     0|
|  ID20|    20|     1|
|  ID72|     1|     0|
|  ID10|     1|     0|
+------+------+------+
only showing top 20 rows



### Ex 2b

In [129]:
!hdfs dfs -rm -r /user/s315054/exam_sim/example_7Feb/ex2b_DF

23/01/31 16:57:15 INFO fs.TrashPolicyDefault: Moved: 'hdfs://BigDataHA/user/s315054/exam_sim/example_7Feb/ex2b_DF' to trash at: hdfs://BigDataHA/user/s315054/.Trash/Current/user/s315054/exam_sim/example_7Feb/ex2b_DF1675184235486


In [130]:
out_2b = '/user/s315054/exam_sim/example_7Feb/ex2b_DF'

# Number of purchases of each item
n_purch_DF = spark.sql("""
        SELECT itemID, sum(CAST(purchased AS int)) AS n_purch
        FROM start_table
        WHERE getYear(timestamp) == 2019
        GROUP BY itemID
""")

n_purch_DF.createOrReplaceTempView('n_purch_tab')

max_purch_DF = spark.sql("""
        SELECT max(n_purch) AS maxTimesPurch
        FROM n_purch_tab
""")

max_purch = max_purch_DF.first().asDict()['maxTimesPurch']

# Open other path
in_path_2 = '/data/students/bigdata_internet/exam_examples_data/exam20200207_data/Items.txt'
items_DF = spark.read.load(in_path_2, format='csv', sep=',', header=False, inferSchema=True)\
                .toDF('itemID', 'name', 'ShortDesc', 'Price', 'webPage')\
                .select('itemID', 'name')

print('helo')

items_DF.createOrReplaceTempView('items_table')

max_items_DF = spark.sql(f"\
                SELECT n_purch_tab.itemID, n_purch_tab.n_purch, items_table.name\
                FROM n_purch_tab, items_table\
                WHERE n_purch_tab.n_purch == {max_purch} AND n_purch_tab.itemID == items_table.itemID\
        ")

max_items_DF.select('name', 'n_purch').write.csv(out_2b, sep=',', header=False)

helo


In [131]:
max_items_DF.show()

+------+-------+--------+
|itemID|n_purch|    name|
+------+-------+--------+
|  ID20|      5|NameID20|
+------+-------+--------+



In [75]:
n_purch_DF.show()

+------+-------+
|itemID|n_purch|
+------+-------+
|  ID62|      0|
|  ID25|      1|
|  ID34|      1|
|  ID54|      0|
|   ID1|      0|
|  ID84|      1|
|  ID23|      0|
|  ID58|      0|
|  ID77|      0|
|  ID35|      0|
|  ID70|      0|
|  ID43|      0|
|  ID89|      0|
|  ID11|      1|
|  ID65|      0|
|  ID28|      0|
|  ID56|      1|
|  ID20|      5|
|  ID72|      1|
|  ID10|      0|
+------+-------+
only showing top 20 rows

